In [210]:
import pandas as pd
import numpy as np

In [223]:
#Read data, grab only the first table, not the date table
ua_file = pd.read_csv('UA_Page.csv', header = 6, thousands=',')
n_values = np.logical_not(ua_file.isnull()).sum(axis=1)
cut_off = n_values[n_values == 0].index[0]
ua_file = ua_file[:cut_off -1]
ua_file['Users'] = ua_file['Users'].str.replace(',','')
g4_file = pd.read_csv('GA4_Page.csv', header = 6, thousands=',')
g4_file = g4_file[['Page path + query string', 'Total users', 'Sessions']]
g4_file.rename(columns={'Page path + query string': 'Page', 'Total users': 'Users'}, inplace = True)

#Add data source column

ua_file['data_source'] = 'UA'
g4_file['data_source'] = 'G4'

#Append data, reset index
data = ua_file.append(g4_file).reset_index()
data.drop(columns={'index'}, inplace = True)
data

,Page,Users,Sessions,data_source
0,/,88649,107744.0,UA
1,/buildyourhonda,84716,70502.0,UA
2,/buildyourhonda/trims,68073,3449.0,UA
3,/vehicles,62600,10231.0,UA
4,/buildyourhonda/colours,48675,8113.0,UA
...,...,...,...,...
54996,/accueil?gclsrc=aw.ds&ds_rl=1006182&ds_rl=1235...,1,1.0,G4
54997,/accueil?gclsrc=aw.ds&ds_rl=1006182&ds_rl=1235...,1,1.0,G4
54998,/accueil?gclsrc=aw.ds&ds_rl=1006182&ds_rl=1235...,1,1.0,G4
54999,/accueil?gclsrc=aw.ds&ds_rl=1006182&ds_rl=1235...,1,1.0,G4


In [212]:
#Create pivot table

piv = pd.pivot_table(data, values = ['Users', 'Sessions'], index = 'Source / Medium', columns = 'data_source', aggfunc = {'Users':sum, 'Sessions':sum})

In [213]:
piv.columns = [c[1] + ' ' +  c[0] for c in piv.columns]

In [214]:
piv['G4 Users'] = piv['G4 Users'].astype('float')
piv['UA Users'] = piv['UA Users'].astype('float')

In [215]:
piv.dtypes

G4 Sessions    float64
UA Sessions    float64
G4 Users       float64
UA Users       float64
dtype: object

In [216]:
piv['Session Difference'] = piv['G4 Sessions'] - piv['UA Sessions']
piv['Session Variance (%)'] = (piv['G4 Sessions'] - piv['UA Sessions'])/ piv['UA Sessions']
piv['Users Difference'] = piv['G4 Users'] - piv['UA Users']
piv['Users Variance (%)'] = (piv['G4 Users'] - piv['UA Users'])/ piv['UA Users']

In [218]:
piv = piv[[ 'G4 Users', 'UA Users', 'Users Difference', 'Users Variance (%)', 'G4 Sessions', 'UA Sessions', 'Session Difference', 'Session Variance (%)']]

In [219]:
piv.sort_values(by = ['G4 Users'], ascending = False, inplace = True)

In [220]:
piv

,G4 Users,UA Users,Users Difference,Users Variance (%),G4 Sessions,UA Sessions,Session Difference,Session Variance (%)
Source / Medium,,,,,,,,
(not set) / (not set),288665.0,2960.0,285705.0,96.521959,9887.0,3948.0,5939.0,1.504306
(direct) / (none),86465.0,51478.0,34987.0,0.679650,110605.0,69490.0,41115.0,0.591668
google / organic,83615.0,93557.0,-9942.0,-0.106267,120074.0,130831.0,-10757.0,-0.082221
google / cpc,44238.0,72195.0,-27957.0,-0.387243,60819.0,102148.0,-41329.0,-0.404599
Numero-Masse / Video,17586.0,NaN,NaN,NaN,26196.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
www-theweathernetwork-com.cdn.ampproject.org / referral,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN
zbexfgvatdybhe_fzbvy / gfkg_cevibdlcbyvdl,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN
zbexfgvatdybhe_fzbvy / gfkg_gfezfbaedbaevgvbaf,NaN,21.0,NaN,NaN,NaN,21.0,NaN,NaN


In [221]:
piv.to_excel('Pivot_Source_Medium.xlsx')

PermissionError: [Errno 13] Permission denied: 'Pivot_Source_Medium.xlsx'